In [4]:
from ccxt import binance,okex5,bitget,bybit,__version__
import pandas as pd
from random import random,randint,uniform
from datetime import datetime

In [5]:
__version__
#2.4.4 -> futures in binance
#3.0.X -> futures in binanceusdm

'2.4.4'

In [6]:
#binance
binance_ex = binance({
    "options": {
        "defaultType": "future",
    }
})
binance_ex.load_markets()  
random_open_order_symbols = [binance_ex.symbols[i] for i in sorted([int(random()*len(binance_ex.symbols)) for i in range(25)])]

In [7]:
binance_ex.load_markets()  

{'BTC/USDT': {'id': 'BTCUSDT',
  'symbol': 'BTC/USDT',
  'base': 'BTC',
  'quote': 'USDT',
  'baseId': 'BTC',
  'quoteId': 'USDT',
  'active': True,
  'type': 'future',
  'linear': True,
  'inverse': False,
  'spot': False,
  'swap': True,
  'future': True,
  'option': False,
  'margin': False,
  'contract': True,
  'contractSize': 1.0,
  'expiry': None,
  'expiryDatetime': None,
  'optionType': None,
  'strike': None,
  'settle': 'USDT',
  'settleId': 'USDT',
  'precision': {'amount': 3, 'price': 1, 'base': 8, 'quote': 8},
  'limits': {'amount': {'min': 0.001, 'max': 1000.0},
   'price': {'min': 556.8, 'max': 4529764.0},
   'cost': {'min': 5.0, 'max': None},
   'leverage': {'min': None, 'max': None},
   'market': {'min': 0.001, 'max': 120.0}},
  'info': {'symbol': 'BTCUSDT',
   'pair': 'BTCUSDT',
   'contractType': 'PERPETUAL',
   'deliveryDate': '4133404800000',
   'onboardDate': '1569398400000',
   'status': 'TRADING',
   'maintMarginPercent': '2.5000',
   'requiredMarginPercent': '

In [3]:
entry_data = {}
for symbol in random_open_order_symbols:
    side = randint(0,1)
    symbol_history_df = pd.DataFrame(binance_ex.fetch_ohlcv(symbol, '15m', limit=1000))
    symbol_max_value, symbol_min_value = binance_ex.markets[symbol]['limits']['price']['max'], binance_ex.markets[symbol]['limits']['price']['min']
    quantity = uniform(symbol_min_value,symbol_max_value)
    if quantity*symbol_history_df[4][1] > 1000:
        quantity = round(1000/symbol_history_df[4][1],4)
    if side == 0:
        most_lowest_price_index = symbol_history_df[4].idxmin()
        entry_data[symbol] = {
            'side': 'buy',
            'entry_price': symbol_history_df[4][most_lowest_price_index],
            'entry_time': datetime.fromtimestamp(symbol_history_df[0][most_lowest_price_index]/1000).strftime("%Y-%m-%d %H:%M:%S"),
            'entry_quantity': quantity,
            'entry_leverage': randint(1,25),
            'entry_type': 'market',
            'entry_class_SF': 'future',
            'exchange': 'binance',
        }
    else:
        most_highest_price_index = symbol_history_df[4].idxmax()
        entry_data[symbol] = {
            'side': 'sell',
            'entry_price': symbol_history_df[4][most_highest_price_index],
            'entry_time': datetime.fromtimestamp(symbol_history_df[0][most_highest_price_index]/1000).strftime("%Y-%m-%d %H:%M:%S"),
            'entry_quantity': quantity,
            'entry_leverage': randint(1,25),
            'entry_type': 'market',
            'entry_class_SF': 'future',
            'exchange': 'binance',
        }

# entry_data #fake data , regenerate it before use
    

In [4]:
def Caculate_Unrealized_Pnl(open_orders_data:dict):
    for symbol in open_orders_data:
        symbol_history_df = pd.DataFrame(binance_ex.fetch_ohlcv(symbol, '1m', limit=10))
        most_recent_price_index = symbol_history_df[4].idxmax()
        most_recent_price = symbol_history_df[4][most_recent_price_index]
        if open_orders_data[symbol]['side'] == 'buy':
            open_orders_data[symbol]['Unrealized'] = round((most_recent_price - open_orders_data[symbol]['entry_price'])*open_orders_data[symbol]['entry_quantity'],2)
        else:
            open_orders_data[symbol]['Unrealized'] = round((open_orders_data[symbol]['entry_price'] - most_recent_price)*open_orders_data[symbol]['entry_quantity'],2)
    return open_orders_data

open_orders_data = Caculate_Unrealized_Pnl(entry_data)
open_orders_data

{'AMB/BUSD': {'side': 'buy',
  'entry_price': 0.013449,
  'entry_time': '2023-03-21 10:00:00',
  'entry_quantity': 7752.514161207281,
  'entry_leverage': 23,
  'entry_type': 'market',
  'entry_class_SF': 'future',
  'exchange': 'binance',
  'Unrealized': 1.67},
 'BLUEBIRD/USDT': {'side': 'buy',
  'entry_price': 6.849,
  'entry_time': '2023-03-12 00:15:00',
  'entry_quantity': 143.0206,
  'entry_leverage': 2,
  'entry_type': 'market',
  'entry_class_SF': 'future',
  'exchange': 'binance',
  'Unrealized': 244.14},
 'BTC/BUSD': {'side': 'buy',
  'entry_price': 19897.2,
  'entry_time': '2023-03-11 03:15:00',
  'entry_quantity': 0.0501,
  'entry_leverage': 23,
  'entry_type': 'market',
  'entry_class_SF': 'future',
  'exchange': 'binance',
  'Unrealized': 404.1},
 'CFX/USDT': {'side': 'buy',
  'entry_price': 0.134,
  'entry_time': '2023-03-12 00:00:00',
  'entry_quantity': 4195.887247038546,
  'entry_leverage': 1,
  'entry_type': 'market',
  'entry_class_SF': 'future',
  'exchange': 'binanc

In [5]:
#data to pandas data frame
open_orders_data = pd.DataFrame(open_orders_data).T
open_orders_data

,side,entry_price,entry_time,entry_quantity,entry_leverage,entry_type,entry_class_SF,exchange,Unrealized
AMB/BUSD,buy,0.013449,2023-03-21 10:00:00,7752.514161,23,market,future,binance,1.67
BLUEBIRD/USDT,buy,6.849,2023-03-12 00:15:00,143.0206,2,market,future,binance,244.14
BTC/BUSD,buy,19897.2,2023-03-11 03:15:00,0.0501,23,market,future,binance,404.1
CFX/USDT,buy,0.134,2023-03-12 00:00:00,4195.887247,1,market,future,binance,1136.25
DAR/USDT,sell,0.2149,2023-03-21 01:15:00,6527.4151,13,market,future,binance,109.66
DOGE/BUSD,sell,0.07871,2023-03-18 10:00:00,1133.914649,17,market,future,binance,7.34
DUSK/USDT,buy,0.14237,2023-03-12 00:00:00,6406.1499,14,market,future,binance,338.89
ENS/USDT,buy,11.521,2023-03-12 00:00:00,80.7494,4,market,future,binance,175.31
GMX/USDT,buy,53.11,2023-03-11 15:30:00,15.873,12,market,future,binance,409.36
ICP/USDT,buy,4.54,2023-03-11 19:30:00,203.2934,11,market,future,binance,121.16
